In [49]:
!pip install gensim

  Using cached gensim-4.2.0-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (24.1 MB)
  Using cached smart_open-6.0.0-py3-none-any.whl (58 kB)
  Using cached scipy-1.9.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (43.4 MB)
  Using cached numpy-1.23.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pythonwhat 2.23.1 requires dill~=0.2.7.1, but you have dill 0.3.3 which is incompatible.
pythonwhat 2.23.1 requires jinja2~=2.10, but you have jinja2 3.0.1 which is incompatible.
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console

In [50]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import pandas as pd

In [51]:
df = pd.read_csv('tweets_translated.csv')

In [52]:
df.dropna(subset=['tweet'],inplace=True)

In [53]:
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))

In [54]:
df['date'] = pd.to_datetime(df['date'])

In [55]:
df = df.sort_values('date')

In [56]:
dic_week = {}
date = df.date.unique()[4]
week = 1
while date in df.date.unique():
    end_week = date + pd.Timedelta(6, 'd')
    dic_week[f'week: {week}'] = (date, end_week)
    week += 1 
    date = end_week + pd.Timedelta(1, 'd')
dic_week

{'week: 1': (numpy.datetime64('2022-07-18T00:00:00.000000000'),
  Timestamp('2022-07-24 00:00:00')),
 'week: 2': (Timestamp('2022-07-25 00:00:00'),
  Timestamp('2022-07-31 00:00:00')),
 'week: 3': (Timestamp('2022-08-01 00:00:00'),
  Timestamp('2022-08-07 00:00:00'))}

In [57]:
for x, y in dic_week.items():
    t = df[(df.date <= y[1]) & (df.date >= y[0])]
    print(t.date.unique())

['2022-07-18T00:00:00.000000000' '2022-07-19T00:00:00.000000000'
 '2022-07-20T00:00:00.000000000' '2022-07-23T00:00:00.000000000'
 '2022-07-24T00:00:00.000000000']
['2022-07-25T00:00:00.000000000' '2022-07-26T00:00:00.000000000'
 '2022-07-27T00:00:00.000000000' '2022-07-28T00:00:00.000000000'
 '2022-07-29T00:00:00.000000000' '2022-07-30T00:00:00.000000000'
 '2022-07-31T00:00:00.000000000']
['2022-08-01T00:00:00.000000000' '2022-08-02T00:00:00.000000000'
 '2022-08-03T00:00:00.000000000' '2022-08-04T00:00:00.000000000'
 '2022-08-05T00:00:00.000000000' '2022-08-06T00:00:00.000000000']


In [58]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/repl/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [59]:
stemmer = SnowballStemmer('english')

In [60]:
no_list = ['live', 'formula', 'stream', 'watch']

In [61]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    
    for token in gensim.utils.simple_preprocess(text):
        #We accept the word max since it is used for driver Max Verstappen
        if (token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and token not in no_list) or token == 'max':
            
            result.append(lemmatize_stemming(token))
            
    return result

In [69]:
df_days = pd.read_csv('topics_day.csv')
df_days = df_days.rename(columns={'Unnamed: 0': 'date'})
df_days.index = df_days.iloc[:,0]
df_days = df_days.drop('date', axis=1)
list(df_days.index[0:-2])

['2022-07-14',
 '2022-07-15',
 '2022-07-16',
 '2022-07-17',
 '2022-07-18',
 '2022-07-19',
 '2022-07-20',
 '2022-07-23',
 '2022-07-24',
 '2022-07-25',
 '2022-07-26',
 '2022-07-27',
 '2022-07-28',
 '2022-07-29',
 '2022-07-30',
 '2022-07-31',
 '2022-08-01',
 '2022-08-02',
 '2022-08-03',
 '2022-08-04']

In [71]:
from gensim import corpora, models
from tqdm import tqdm
day_topics = {}

for day in tqdm(df.date.unique()):
    just_day = str(day).split('T')[0]
    if just_day not in list(df_days.index[:-2]):
        topics_per_day = {}
        day_df = df[df.date == day]
        processed = day_df['tweet'].map(preprocess)
        dictionary = gensim.corpora.Dictionary(processed)

        # filter words that abear in less than 20% off tweets and keep all popular words
        dictionary.filter_extremes(no_below=0.2, no_above=0.45, keep_n=100000)

        # Create a Bag-of-Words model fot every tweet
        bow_corpus = [dictionary.doc2bow(tweet) for tweet in processed]

        # creating a TF_IDF model
        tfidf = models.TfidfModel(bow_corpus)

        # Apply transformation to the entire corpus call
        corpus_tfidf = tfidf[bow_corpus]

        # LDA modeling 
        lda_model = gensim.models.LdaMulticore(corpus_tfidf,
                                             num_topics=3,
                                             id2word=dictionary,
                                             passes=2)


        for idx, topic in lda_model.print_topics(-1):
            topics_per_day[idx] = topic

        day_topics[just_day] = topics_per_day

  9%|▉         | 2/22 [00:10<01:43,  5.18s/it]Process ForkPoolWorker-36:
Process ForkPoolWorker-44:
Process ForkPoolWorker-37:
Process ForkPoolWorker-42:
Process ForkPoolWorker-34:
Process ForkPoolWorker-38:
Process ForkPoolWorker-43:
Process ForkPoolWorker-33:
Process ForkPoolWorker-41:
Process ForkPoolWorker-32:
Process ForkPoolWorker-40:
Process ForkPoolWorker-35:
Process ForkPoolWorker-31:
Process ForkPoolWorker-45:
Process ForkPoolWorker-39:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in 

In [72]:
day_topics

{'2022-08-05': {0: '0.006*"mclaren" + 0.005*"ricciardo" + 0.005*"race" + 0.004*"daniel" + 0.004*"piastri" + 0.004*"like" + 0.004*"driver" + 0.003*"lmer" + 0.003*"mercedesamgf" + 0.003*"know"',
  1: '0.006*"team" + 0.005*"news" + 0.004*"blog_formula" + 0.003*"ricciardo" + 0.003*"mercedesamgf" + 0.003*"charles_leclerc" + 0.003*"motorsport" + 0.003*"piastri" + 0.003*"alpin" + 0.003*"ferrari"',
  2: '0.008*"blog_formula" + 0.007*"mercedesamgf" + 0.004*"max" + 0.004*"redbullrac" + 0.004*"year" + 0.004*"verstappen" + 0.003*"ferrari" + 0.003*"driver" + 0.003*"mclaren" + 0.003*"topp"'},
 '2022-08-06': {0: '0.011*"redbul" + 0.004*"redbullrac" + 0.004*"team" + 0.004*"fiaformula" + 0.004*"twommof" + 0.004*"news" + 0.003*"race" + 0.003*"ferrari" + 0.003*"like" + 0.003*"think"',
  1: '0.004*"race" + 0.004*"news" + 0.003*"challeng" + 0.003*"mclaren" + 0.003*"sport" + 0.003*"blog_formula" + 0.003*"fiaformula" + 0.003*"twommof" + 0.003*"driver" + 0.003*"reddit"',
  2: '0.008*"blog_formula" + 0.004*"li

In [75]:
day_df = pd.DataFrame.from_dict(day_topics)
day_df = day_df.transpose()
topics=[f'Topic: {n}' for n in range(1, len(day_df.columns)+1)]
day_df.columns = topics
day_df = pd.concat([df_days, day_df])

In [76]:
day_df

,Topic: 1,Topic: 2,Topic: 3
2022-07-14,"0.004*""motorsport"" + 0.004*""grandprix"" + 0.004...","0.004*""like"" + 0.003*""lewishamilton"" + 0.003*""...","0.008*""silverston"" + 0.004*""like"" + 0.004*""blo..."
2022-07-15,"0.019*""silverston"" + 0.010*""saudi"" + 0.007*""ar...","0.006*""race"" + 0.005*""tornello"" + 0.004*""leof""...","0.007*""blog_formula"" + 0.004*""news"" + 0.003*""r..."
2022-07-16,"0.019*""alpinef"" + 0.019*""oconesteban"" + 0.018*...","0.007*""blog_formula"" + 0.005*""silverston"" + 0....","0.004*""blog_formula"" + 0.004*""like"" + 0.003*""d..."
2022-07-17,"0.008*""naturalparadigm"" + 0.006*""race"" + 0.005...","0.006*""blog_formula"" + 0.004*""jule"" + 0.004*""r...","0.008*""sunday"" + 0.005*""driver"" + 0.004*""like""..."
2022-07-18,"0.019*""gpfrancef"" + 0.007*""blog_formula"" + 0.0...","0.007*""gpfrancef"" + 0.004*""hamilton"" + 0.004*""...","0.014*""landonorri"" + 0.013*""gpfrancef"" + 0.005..."
2022-07-19,"0.006*""race"" + 0.004*""hamilton"" + 0.004*""drive...","0.008*""blog_formula"" + 0.005*""ocon"" + 0.004*""y...","0.005*""news"" + 0.005*""like"" + 0.004*""verstappe..."
2022-07-20,"0.007*""scuderiaferrari"" + 0.005*""team"" + 0.004...","0.005*""race"" + 0.004*""nyckdevri"" + 0.004*""driv...","0.005*""tomek_poradzisz"" + 0.005*""sauberesport""..."
2022-07-23,"0.016*""mercedesamgf"" + 0.016*""lewishamilton"" +...","0.008*""charles_leclerc"" + 0.006*""landonorri"" +...","0.010*""scuderiaferrari"" + 0.007*""pole"" + 0.007..."
2022-07-24,"0.019*""french"" + 0.014*""stream"" + 0.014*""race""...","0.020*""𝐋𝐈𝐕𝐄"" + 0.018*""mercedesamgf"" + 0.017*""l...","0.018*""link"" + 0.018*""grand"" + 0.018*""prix"" + ..."
2022-07-25,"0.010*""max"" + 0.010*""verstappen"" + 0.008*""redb...","0.006*""redbullrac"" + 0.006*""max"" + 0.006*""lewi...","0.008*""team"" + 0.007*""alpinef"" + 0.006*""race"" ..."


In [ ]:
day_df.to_csv('topics_day.csv')

In [19]:
df_date = df[df.date == '2022-07-28']
processed = df_date['tweet'].map(preprocess)

In [20]:
processed = df['tweet'].map(preprocess)

In [21]:
processed[:10]

233131                                            [maxpapi]
233006    [atellodemenes, schecoperez, redbullrac, prime...
233007    [santander_ar, challeng, entrepreneur, world, ...
233008                         [life, go, like, verstappen]
233009                   [phoenixownsyou, paulo, interlago]
233010    [atellodemenes, schecoperez, redbullrac, prime...
233011                                    [crowd, complain]
233012    [diegocanoborroy, aaron_martinzgz, button, wor...
233013         [better, wors, michael, masi, video, youtub]
233014                                        [meunomeesaf]
Name: tweet, dtype: object

In [22]:
dictionary = gensim.corpora.Dictionary(processed)

In [23]:
# filter words that abear in less than 20% off tweets and keep all popular words
dictionary.filter_extremes(no_below=0.2, no_above=0.5, keep_n=100000)

In [24]:
# Create a Bag-of-Words model fot every tweet
bow_corpus = [dictionary.doc2bow(tweet) for tweet in processed]

In [25]:
# creating a TF_IDF model
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
print(tfidf)

TfidfModel<num_docs=263441, num_nnz=1730932>


In [26]:
# Apply transformation to the entire corpus call
corpus_tfidf = tfidf[bow_corpus]
print(corpus_tfidf[0])

[(0, 1.0)]


In [27]:
from pprint import pprint

for tweet in corpus_tfidf:
    pprint(tweet)
    break

[(0, 1.0)]


In [ ]:
# LDA modeling 
lda_model = gensim.models.LdaMulticore(bow_corpus,
                                     num_topics=10,
                                     id2word=dictionary,
                                     alpha=0.75,
                                     eta=0.5,
                                     passes=2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print(f'Topics {idx} \nWords: {topic}')
    print('\n')

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf,
                                     num_topics=10,
                                     id2word=dictionary,
                                     passes=2)

In [ ]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print(f'Topics {idx} \nWords: {topic}')
    print('\n')